# 看護師のシフトの最適化

## 0. ライブラリのインストール（ガントチャート作成用）
インストール後 カーネル再起動

In [1]:
!pip install plotly --upgrade

     |████████████████████████████████| 7.1MB 26.7MB/s eta 0:00:01
  Found existing installation: plotly 3.6.1
    Uninstalling plotly-3.6.1:
      Successfully uninstalled plotly-3.6.1


## 1. データの準備

In [1]:
!git clone https://github.com/IBMDecisionOptimization/DOforDSX-Nurses-example.git

Cloning into 'DOforDSX-Nurses-example'...
remote: Enumerating objects: 72, done.
remote: Total 72 (delta 0), reused 0 (delta 0), pack-reused 72
Unpacking objects: 100% (72/72), done.


In [2]:
import pandas as pd

In [3]:
DPATH='DOforDSX-Nurses-example/datasets/'

df_skills = pd.read_csv(DPATH +  'Skills.csv')
df_depts = pd.read_csv(DPATH + 'Departments.csv')
df_shifts = pd.read_csv(DPATH + 'Shifts.csv')
df_shifts.index.name = 'shiftId'
df_nurses = pd.read_csv(DPATH + 'Nurses.csv')
df_nurse_skills = pd.read_csv(DPATH + 'NurseSkills.csv')
df_vacations = pd.read_csv(DPATH + 'NurseVacations.csv')
df_associations = pd.read_csv(DPATH + 'NurseAssociations.csv')
df_incompatibilities = pd.read_csv(DPATH + 'NurseIncompatibilities.csv')
df_requirements = pd.read_csv(DPATH + 'SkillsRequirements.csv')

In [4]:
# Display the nurses dataframe
print("#nurses = {}".format(len(df_nurses)))
print("#shifts = {}".format(len(df_shifts)))

#nurses = 32
#shifts = 36


#### 32人の看護師を１週間の36シフトに割り当てる問題になります

In [5]:
from collections import namedtuple

In [6]:
_all_days = ["monday",
             "tuesday",
             "wednesday",
             "thursday",
             "friday",
             "saturday",
             "sunday"]


def day_to_day_week(day):
    day_map = {day: d for d, day in enumerate(_all_days)}
    return day_map[day.lower()]

In [7]:
class TNurse(namedtuple("TNurse1", ["name", "seniority", "qualification", "pay_rate"])):
    def __str__(self):
        return self.name

In [8]:
class TShift(namedtuple("TShift",
                        ["department", "day", "start_time", "end_time", "min_requirement", "max_requirement"])):

    def __str__(self):
        # keep first two characters in department, uppercase
        dept2 = self.department[0:4].upper()
        # keep 3 days of weekday
        dayname = self.day[0:3]
        return '{}_{}_{:02d}'.format(dept2, dayname, self.start_time).replace(" ", "_")

In [9]:
class ShiftActivity(object):
    @staticmethod
    def to_abstime(day_index, time_of_day):
        """ Convert a pair (day_index, time) into a number of hours since Monday 00:00

        :param day_index: The index of the day from 1 to 7 (Monday is 1).
        :param time_of_day: An integer number of hours.

        :return:
        """
        time = 24 * (day_index - 1)
        time += time_of_day
        return time

    def __init__(self, weekday, start_time_of_day, end_time_of_day):
        assert (start_time_of_day >= 0)
        assert (start_time_of_day <= 24)
        assert (end_time_of_day >= 0)
        assert (end_time_of_day <= 24)

        self._weekday = weekday
        self._start_time_of_day = start_time_of_day
        self._end_time_of_day = end_time_of_day
        # conversion to absolute time.
        start_day_index = day_to_day_week(self._weekday)
        self.start_time = self.to_abstime(start_day_index, start_time_of_day)
        end_day_index = start_day_index if end_time_of_day > start_time_of_day else start_day_index + 1
        self.end_time = self.to_abstime(end_day_index, end_time_of_day)
        assert self.end_time > self.start_time

    @property
    def duration(self):
        return self.end_time - self.start_time

    def overlaps(self, other_shift):
        if not isinstance(other_shift, ShiftActivity):
            return False
        else:
            return other_shift.end_time > self.start_time and other_shift.start_time < self.end_time

In [10]:
shifts = [TShift(*shift_row) for shift_row in df_shifts.itertuples(name=None, index=False)]
nurses = [TNurse(*nurse_row) for nurse_row in df_nurses.itertuples(name=None, index=False)]
shift_activities = {s: ShiftActivity(s.day, s.start_time, s.end_time) for s in shifts}
nurses_by_id = {n.name: n for n in nurses}

## 2. モデルの作成

### モデルの宣言

In [11]:
from docplex.mp.model import Model
mdl = Model("Nurses")

### 決定変数の定義
- 決定変数は看護師（32人）とシフト（36コマ）のマトリックスで定義します
- 後続の制約、目的関数を満たすために、32x36=1152個の変数に1(勤務)あるいは0(休暇)を決める問題ということになります。

In [12]:
nurse_assignment_vars = mdl.binary_var_matrix(nurses, shifts, 'NurseAssigned')

## 制約の定義

#### 制約1: 最長労働時間
各看護師の１週間の労働時間を40時間以下にします。

In [13]:
max_work_time = 40

for n in nurses:
    mdl.add_constraint(
        mdl.sum(nurse_assignment_vars[n,s]*shift_activities[s].duration for s in shifts) <= max_work_time,
        "max_time_{0!s}".format(n)
    )

#### 制約2: 休暇
Vacation表の定義の通り、各看護師の休暇の日は勤務できないものとします。

In [14]:
for vac_nurse_id, vac_day in df_vacations.itertuples(name=False, index=False):
    vac_n = nurses_by_id[vac_nurse_id]
    for shift in (s for s in shifts if s.day == vac_day):
        mdl.add_constraint(
            nurse_assignment_vars[vac_n, shift] == 0,
            "medium_vacations_{0!s}_{1!s}_{2!s}".format(vac_n, vac_day, shift)
        )

#### 制約3: 重複シフト
シフトによっては、前後のシフトと時間が重複しているものがあるので、重複しているシフトに同一の看護師は割り当て不可とします。

In [15]:
for i1 in range(len(shifts)):
    for i2 in range(i1+1, len(shifts)):
        s1 = shifts[i1]
        s2 = shifts[i2]
        if shift_activities[s1].overlaps(shift_activities[s2]):
            for n in nurses:
                mdl.add_constraint(
                    nurse_assignment_vars[n,s1]+nurse_assignment_vars[n,s2]<=1,
                    "high_overlapping_{0!s}_{1!s}_{2!s}".format(s1,s2,n)
                )

#### 制約4: シフト毎の必要人数
各シフト毎に定義された最小/最大の人数を満たす看護師を割り当てます。

In [16]:
for s in shifts:
    demand_min = s.min_requirement
    demand_max = s.max_requirement
    
    total_assigned = mdl.sum(nurse_assignment_vars[n,s] for n in nurses)
    mdl.add_constraint(total_assigned >= demand_min, "high_req_min_{0!s}_{1}".format(s, demand_min))
    mdl.add_constraint(total_assigned <= demand_max, "medium_req_max_{0!s}_{1}".format(s, demand_max))

#### 制約5: シフト毎の必要スキル
シフト毎に定義された必要スキルを保有する看護師を割り当てます。

In [17]:
for (dept,skill,required) in df_requirements.itertuples(name=False,index=False):
    if required > 0:
        for dsh in (s for s in shifts if dept == s.department):
            mdl.add_constraint(
                mdl.sum(nurse_assignment_vars[skilled_nurse, dsh] for skilled_nurse in
                        (n for n in nurses if n.name in df_nurse_skills['nurse'].tolist() and skill in df_nurse_skills[df_nurse_skills['nurse']==n.name]['skill'].tolist())) >= required,
                "high_required_{0!s}_{1!s}_{2!s}_{3!s}".format(dept, skill, required, dsh)
            )

#### 制約6: 特定の看護師同士を同一シフトに割り当て
特定のペアの看護師を同一のシフトに割り当てます。（新人とその指導者等）

In [18]:
for (nurse_id1, nurse_id2) in df_associations.itertuples(name=False,index=False):
    nurse1 = nurses_by_id[nurse_id1]
    nurse2 = nurses_by_id[nurse_id2]
    for s in shifts:
        mdl.add_constraint(
            nurse_assignment_vars[nurse1, s]==nurse_assignment_vars[nurse2, s],
            'medium_ct_nurse_incompat_{0!s}_{1!s}'.format(nurse_id1, nurse_id2)
        )

#### 制約7: 特定の看護師同士を別シフトに割り当て
特定のペアの看護師を別シフトに割り当てます。（相性等?）

In [19]:
for (nurse_id1, nurse_id2) in df_incompatibilities.itertuples(name=False,index=False):
    nurse1 = nurses_by_id[nurse_id1]
    nurse2 = nurses_by_id[nurse_id2]
    for s in shifts:
        mdl.add_constraint(
            nurse_assignment_vars[nurse1,s]+nurse_assignment_vars[nurse2, s] <= 1,
            'medium_ct_nurse_incompat_{0!s}_{1!s}'.format(nurse_id1, nurse_id2)
        )

## 目的関数の定義
１週間の看護師の総コストを最小化することを目的関数とします。

In [20]:
def assignment_cost_f(ns):
    n, s = ns
    return n.pay_rate * shift_activities[s].duration

nurse_costs = mdl.scal_prod_f(nurse_assignment_vars, assignment_cost_f)
total_salary_costs = mdl.sum(nurse_costs)

In [21]:
mdl.add_kpi(total_salary_costs, "Total salary cost")

DecisionKPI(name=Total salary cost,expr=150NurseAssigned_Anne_EMER_Mon_02+100NurseAssigned_Anne_EMER_Mon..)

In [22]:
mdl.minimize(total_salary_costs)

In [23]:
mdl.print_information()

Model: Nurses
 - number of variables: 1152
   - binary=1152, integer=0, continuous=0
 - number of constraints: 1221
   - linear=1221
 - parameters: defaults
 - problem type is: MILP


## 問題を解く

In [24]:
sol=mdl.solve(log_output=True)

CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 2
Tried aggregator 2 times.
MIP Presolve eliminated 657 rows and 336 columns.
MIP Presolve modified 37 coefficients.
Aggregator did 36 substitutions.
Reduced MIP has 528 rows, 780 columns, and 3197 nonzeros.
Reduced MIP has 780 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (4.24 ticks)
Found incumbent of value 27352.000000 after 0.03 sec. (11.06 ticks)
Probing time = 0.00 sec. (0.32 ticks)
Tried aggregator 1 time.
Reduced MIP has 528 rows, 780 columns, and 3197 nonzeros.
Reduced MIP has 780 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (2.11 ticks)
Probing time = 0.00 sec. (0.32 ticks)
Clique table members: 402.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 2 threads.
Root relaxation solution time = 0.01 sec. (3.29 ticks)

        Nodes                 

In [25]:
mdl.report_kpis()

*  KPI: Total salary cost = 25102.000


## 3. 可視化
１週間のシフトをガントチャートで表示します。

In [29]:
import datetime
import plotly.figure_factory as ff
import plotly.offline as offline
offline.init_notebook_mode(connected=True)

In [30]:
df = []
d1 = datetime.datetime(year=2020, month=3, day=2, hour=0)
for n in sorted(nurses):
    total_hours = sum(nurse_assignment_vars[n, s].solution_value * shift_activities[s].duration for s in shifts)
    for s in shifts:
        if nurse_assignment_vars[n, s].solution_value == 1:
            start_time = d1 + datetime.timedelta(days=day_to_day_week(s.day), hours=s.start_time)
            end_time = d1 + datetime.timedelta(days=day_to_day_week(s.day), hours=s.end_time if s.end_time!=2 else 26)
            task = dict(Task=n.name, Start=start_time, Finish=end_time, Resource=s.department)
            df.append(task)

In [31]:
colors = {
    'Emergency': 'rgb(220, 0, 0)',
    'Consultation': 'rgb(0, 255, 100)'
}

fig = ff.create_gantt(df, colors=colors, index_col='Resource', show_colorbar=True,group_tasks=True)
#fig.show()
offline.iplot(fig)

In [32]:
from plotly.offline import plot

In [33]:
fig = ff.create_gantt(df)
plot_fig = plot(fig, output_type='div', include_plotlyjs=False)
print(plot_fig)

<div>
        
        
            <div id="925bb2be-b892-435e-b0a7-cc343d54f162" class="plotly-graph-div" style="height:600px; width:100%;"></div>
            <script type="text/javascript">
                
                    window.PLOTLYENV=window.PLOTLYENV || {};
                    
                if (document.getElementById("925bb2be-b892-435e-b0a7-cc343d54f162")) {
                    Plotly.newPlot(
                        '925bb2be-b892-435e-b0a7-cc343d54f162',
                        [{"fill": "toself", "fillcolor": "rgb(127, 127, 127)", "hoverinfo": "name", "legendgroup": "rgb(127, 127, 127)", "mode": "none", "name": "Zoe", "type": "scatter", "x": ["2020-03-02T12:00:00", "2020-03-02T18:00:00", "2020-03-02T18:00:00", "2020-03-02T12:00:00", "2020-03-02T12:00:00", "2020-03-08T20:00:00", "2020-03-09T02:00:00", "2020-03-09T02:00:00", "2020-03-08T20:00:00", "2020-03-08T20:00:00", "2020-03-02T08:00:00", "2020-03-02T12:00:00", "2020-03-02T12:00:00", "2020-03-02T08:00:00", "2020-